In [1]:
from utils.tools import dotdict
import torch
from exp.exp_main import Exp_Main
import pandas as pd
import shutil
import os

In [2]:
args = dotdict()

args.model = 'FEDformer' # model of experiment, options: [informer, informerstack, informerlight(TBD)]
args.task_id = '2test'
args.is_training = 1
args.version = 'Fourier'
args.task = 'test'
args.data = 'custom' # data
args.root_path = './dataset/' # root path of data file
args.data_path = 'Informer-data.csv' # data file
args.features = 'M' # forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate
args.target = 'Austria' # target feature in S or MS task
args.continent = 'Europe'
args.freq = 'm' # freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h
args.detail_freq = args.freq
args.freq = args.freq[-1:]
args.checkpoints = './informer_checkpoints' # location of model checkpoints

args.mode_select = 'random'
args.modes = 32
args.seq_len = 32 # input sequence length of Informer encoder
args.label_len = 16# start token length of Informer decoder
args.pred_len = 1 # prediction sequence length
# Informer decoder input: concat[start token series(label_len), zero padding series(pred_len)]


args.enc_in = 97 # encoder input size
args.dec_in = 97 # decoder input size
args.c_out  = 97 # output size
args.factor = 2 # probsparse attn factor
args.d_model = 512 # dimension of model
args.n_heads = 8 # num of heads
args.e_layers = 2 # num of encoder layers
args.d_layers = 1 # num of decoder layers
args.d_ff = 2048 # dimension of fcn in model
args.dropout = 0.05 # dropout
args.attn = 'prob' # attention used in encoder, options:[prob, full]
args.embed = 'timeF' # time features encoding, options:[timeF, fixed, learned]
args.activation = 'gelu' # activation
args.distil = True # whether to use distilling in encoder
args.output_attention = False # whether to output attention in ecoder
args.mix = True
args.padding = 0
args.do_predict = True
args.moving_avg = 12
args.cross_activation = 'tanh'
args.base = 'legendre'

args.batch_size = 4
args.learning_rate = 0.0001
args.loss = 'mse'
args.lradj = 'type1'
args.use_amp = False # whether to use automatic mixed precision training

args.num_workers = 0
args.itr = 1
args.train_epochs = 250
args.patience = 5
args.des = 'exp'

args.use_gpu = True if torch.cuda.is_available() else False
args.gpu = 0

args.use_multi_gpu = False
args.devices = '0,1,2,3'

args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False



In [3]:
print(args.is_training)

1


In [ ]:
Exp = Exp_Main

if args.is_training:
    print('hej')
    for m in ['FEDformer']:
        args.model = m
        for b in [8]:
            args.batch_size = b
            for i in range(14, 238):
                    # Set augments by using data name
                    args.idx = i
                    j = str(i)
                    while(len(j)<3):
                        j = '0' + j
                    for ii in range(args.itr):
                        # setting record of experiments
                        setting = '{}_{}_{}_{}_batchsize{}_modes{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
                            j,
                            args.task_id,
                            args.model,
                            args.batch_size,
                            args.mode_select,
                            args.modes,
                            args.data,
                            args.features,
                            args.seq_len,
                            args.label_len,
                            args.pred_len,
                            args.d_model,
                            args.n_heads,
                            args.e_layers,
                            args.d_layers,
                            args.d_ff,
                            args.factor,
                            args.embed,
                            args.distil,
                            args.des,
                            ii)

                        exp = Exp(args)  # set experiments
                        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
                        exp.train(setting)

                        if args.do_predict:
                            print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
                            exp.predict(setting, True)
                            for i in os.listdir("C:/Users/osahl/Desktop/UNI/master thesis/FEDFormer/informer_checkpoints/"):
                                shutil.rmtree('C:/Users/osahl/Desktop/UNI/master thesis/FEDFormer/informer_checkpoints/' + i)

                        torch.cuda.empty_cache()

hej
Use GPU: cuda:0
fourier enhanced block used!
modes=32, index=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
fourier enhanced block used!
modes=32, index=[0, 1, 2, 3, 4, 5, 6, 7]
 fourier enhanced cross attention used!
modes_q=8, index_q=[0, 1, 2, 3, 4, 5, 6, 7]
modes_kv=16, index_kv=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
enc_modes: 16, dec_modes: 8
>>>>>>>start training : 014_2test_FEDformer_8_batchsizerandom_modes32_custom_ftM_sl32_ll16_pl1_dm512_nh8_el2_dl1_df2048_fc2_ebtimeF_dtTrue_exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 184
val 24
test 0
Epoch: 1 cost time: 7.883534669876099
Epoch: 1, Steps: 23 | Train Loss: 1.2875114 Vali Loss: 0.9432063 Test Loss: nan
Validation loss decreased (inf --> 0.943206).  Saving model ...
Updating learning rate to 0.0001
Epoch: 2 cost time: 2.5935378074645996
Epoch: 2, Steps: 23 | Train Loss: 0.9561442 Vali Loss: 0.8917972 Test Loss: nan
Validation loss decreased (0.943206 --> 0.891797).  Saving model ...
Updating learning rate 

KeyboardInterrupt: 

In [ ]:
pd.read_csv(args.root_path + args.data_path)

,date,Austria,Burundi,Belgium,Burkina Faso,Bahamas,Bolivia,Brazil,Barbados,Botswana,...,Uruguay,United States,Samoa,South Africa,Europe,Africa,North America,South America,Asia & Oceania,Global Factor
0,1980-02-01,0.872943,-1.005034,0.753035,-0.435968,0.824407,0.374450,4.513792,-0.203252,3.546852,...,4.302900,1.403974,3.975538,0.985237,0.992071,1.302618,2.107086,1.835498,3.843790,2.017661
1,1980-03-01,0.337436,0.201816,0.182826,2.454607,0.926185,1.646262,5.862831,1.714616,0.526594,...,1.950554,1.509454,1.875400,0.975625,0.764569,1.295151,1.684490,1.933483,0.927116,1.091811
2,1980-04-01,0.432173,1.202419,0.127767,-0.534332,1.025097,0.861332,5.151259,0.895974,0.104987,...,2.533744,1.117331,5.112935,0.966198,1.321007,0.630528,1.128067,2.299137,1.167593,1.133593
3,1980-05-01,0.478009,0.298359,0.291456,-1.348456,0.642057,2.978359,5.546105,0.395648,-0.210084,...,3.410773,0.982813,0.517868,0.956931,0.576158,1.150624,1.085661,2.255788,0.751945,0.828821
4,1980-06-01,1.090843,-0.198807,0.236171,0.838183,0.849713,5.378239,5.169338,2.051855,2.594852,...,3.567083,1.094237,2.348245,1.886842,0.530069,1.757311,1.039153,1.731945,0.823665,0.916747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,2019-08-01,0.093949,0.818558,-0.018337,-1.503201,-0.386421,0.500315,0.109999,1.921638,0.000000,...,0.880221,-0.004989,-0.081060,0.219975,0.099504,1.664324,0.238937,0.279448,0.519181,0.810713
475,2019-09-01,0.468424,0.901719,-0.459999,-0.206744,0.303742,-0.081468,-0.039978,0.972621,0.098280,...,0.514474,0.078310,0.082406,0.329139,0.129016,1.293934,0.186667,0.220231,0.216114,0.607386
476,2019-10-01,0.186757,0.537154,0.358990,0.540568,-0.451343,0.448842,0.100039,0.406711,0.098184,...,0.749476,0.228301,-0.185927,0.000000,0.177487,0.958882,0.322159,0.476384,0.510649,0.592476
477,2019-11-01,0.186409,0.711747,0.064324,-0.747741,0.009231,1.108687,0.508696,0.506074,0.098087,...,0.419433,-0.053617,-0.119933,0.109446,-0.024420,1.475937,0.425120,0.265014,0.250176,0.707107
